In [2]:
%load_ext autoreload
%autoreload 2

from pathlib import Path

import pandas as pd
import scikit_posthocs as sp
import seaborn as sns
import matplotlib.pyplot as plt

from ff_energy.plot import plot_energy_MSE, plot_ff_fit
from ff_energy.structure import atom_key_pairs
from ff_energy.potential import LJ, akp_indx
from ff_energy.ff import FF
from ff_energy.ff_fit import LJ_bound, load_ff, fit_func, fit_repeat
from ff_energy.data import pairs_data
from ff_energy.utils import pickle_output, read_from_pickle, str2int



#  Data 
##  ab intio data

In [3]:
pc_pbe0_dz_d4 = next(read_from_pickle("data_pc_pbe0_d4.obj.pkl"))
pc_pbe0_dz = next(read_from_pickle("data_pc_pbe0.obj.pkl"))
pc_hf_dz = next(read_from_pickle("data_pc_hfdz.obj.pkl"))
pc_pbe0tz = next(read_from_pickle("data_pc_pbe0tz.obj.pkl"))
pnolccsd_pvtzdf = next(read_from_pickle("data_pc_pno-lccsd-pvtzdf.obj.pkl"))

In [4]:
pc_pbe0_dz.bonded_fit.__dict__

{'df':             m_ENERGY         KEY      key  monomer           a        r1  \
 test467_7   1.045872   test467_7  test467        7  100.423002  0.992102   
 test467_16  0.771207  test467_16  test467       16  103.196331  0.929730   
 test467_15  0.472133  test467_15  test467       15  105.643400  0.969068   
 test467_1   2.142709   test467_1  test467        1   94.337119  0.995515   
 test467_19  0.853542  test467_19  test467       19  103.553318  0.988386   
 ...              ...         ...      ...      ...         ...       ...   
 test66_18   0.304634   test66_18   test66       18  100.073321  0.964746   
 test66_15   4.186819   test66_15   test66       15   90.670120  1.012864   
 test66_20   3.384820   test66_20   test66       20   93.650798  1.011076   
 test66_2    4.564369    test66_2   test66        2   95.501413  1.042403   
 test66_9    1.258765    test66_9   test66        9  105.405358  0.937973   
 
                   r2    E_pred        SE  
 test467_7   0.988528  0

In [376]:
np.rad2deg(1.8203314482389932)

104.29730929903118

In [129]:
pc_pbe0_dz.data

,ECOL,KEY,TOTAL,ELEC,VDW,KEY,M_ENERGY,KEY,n_monomers,C_ENERGY,n_pairs,intE,P_intE,m_E_tot,p_m_E_tot,C_ENERGY_kcalmol,NBONDS,nb_intE,n,epol_mike
test0,-173.022645,test0,24.14023,-86.50353,20.41138,test0,-1527.133263,test0,20,-1527.276175,190,-89.677734,-77.478222,-958280.703813,-958281.958376,-958370.381547,-66.09215,-66.09215,0,-197.910658
test1,-105.096240,test1,25.25058,-55.44306,8.97296,test1,-1527.135387,test1,20,-1527.221635,189,-54.121080,-53.285800,-958282.036711,-958282.584930,-958336.157791,-46.47010,-46.47010,1,-113.959761
test2,-144.567121,test2,24.75756,-76.78875,14.41942,test2,-1527.140946,test2,20,-1527.260418,190,-74.969007,-70.885989,-958285.524837,-958285.865743,-958360.493844,-62.36933,-62.36933,2,-160.484782
test3,-105.558619,test3,21.75948,-60.46748,5.74741,test3,-1527.151295,test3,20,-1527.247454,190,-60.340318,-58.662892,-958292.019016,-958292.985606,-958352.359334,-54.72007,-54.72007,3,-118.662845
test4,-141.957058,test4,21.69034,-77.22292,12.55821,test4,-1527.150321,test4,20,-1527.270835,190,-75.623009,-73.371184,-958291.407609,-958292.418141,-958367.030618,-64.66471,-64.66471,4,-155.773808
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
test495,-156.403481,test495,24.92171,-83.61850,17.54292,test495,-1527.129876,test495,20,-1527.255655,190,-78.926606,-72.611238,-958278.578831,-958280.757990,-958357.505436,-66.07558,-66.07558,495,-175.806320
test496,-126.839243,test496,15.65968,-71.27947,12.74644,test496,-1527.168887,test496,20,-1527.264530,190,-60.016740,-61.118914,-958303.057842,-958304.589708,-958363.074582,-58.53303,-58.53303,496,-138.410126
test497,-160.377711,test497,20.65947,-76.28248,24.76786,test497,-1527.154064,test497,20,-1527.248923,190,-59.524062,-57.960443,-958293.756804,-958294.973787,-958353.280867,-51.51462,-51.51462,497,-178.929213
test498,-154.210967,test498,21.38907,-73.28759,20.35679,test498,-1527.150078,test498,20,-1527.257280,190,-67.269733,-65.470853,-958291.255270,-958293.538908,-958358.525003,-52.93080,-52.93080,498,-171.903376


In [137]:
pc_pbe0_dz.pairs_df.sort_values("pair")

,p_ENERGY,p_m1_ENERGY,p_m2_ENERGY,p_int_ENERGY,key,pair
test283_1_2,-152.716531,-76.357006,-76.355244,-0.004281,test283,"(1, 2)"
test458_1_2,-152.715374,-76.355728,-76.359576,-0.000069,test458,"(1, 2)"
test55_1_2,-152.714443,-76.356881,-76.358395,0.000833,test55,"(1, 2)"
test312_1_2,-152.711803,-76.357556,-76.354086,-0.000162,test312,"(1, 2)"
test131_1_2,-152.713646,-76.356216,-76.357370,-0.000060,test131,"(1, 2)"
...,...,...,...,...,...,...
test383_19_20,-152.716114,-76.357536,-76.358707,0.000129,test383,"(19, 20)"
test89_19_20,-152.715109,-76.356613,-76.358461,-0.000034,test89,"(19, 20)"
test213_19_20,-152.716230,-76.358613,-76.357787,0.000170,test213,"(19, 20)"
test335_19_20,-152.716469,-76.357878,-76.358238,-0.000354,test335,"(19, 20)"


In [55]:
pair_eel_epol_path = "/home/boittier/homeb/pair-eel-epol"
files = list(Path(pair_eel_epol_path).glob("*.dat"))

In [145]:
import jax.numpy as jnp
import numpy as np
pair_eel_epol = np.zeros((500,190))
pair_eel_epol.shape

(500, 190)

In [147]:
for i in range(len(files)):
    fi = files[i]
    IDX = str2int(fi.name.split("-")[0])
    with open(fi) as f:
        for j, line in enumerate(f.readlines()):
            # print(j, line)
            a,b,E = line.split()
            pair_eel_epol[i,j] = float(E)
            

In [196]:
pc_pbe0_dz.coloumb["ECOL"].loc["test0_1_2.py"]

-0.6380645095033973

In [220]:
# pair_eel_epol.sum(axis=1)

In [167]:
test_ff.dcm_ecols

Array([ 3.340709  ,  1.6438198 , -4.480503  , ..., -1.8386152 ,
        1.2585741 ,  0.63554394], dtype=float32)

In [168]:
test_ff.dcm_pairs_idx

Array([[[ 4, 16]],

       [[ 4, 16]],

       [[ 4, 16]],

       ...,

       [[ 1, 16]],

       [[ 1, 16]],

       [[ 1, 16]]], dtype=int32)

In [178]:
cond_mask = np.where(test_ff.cluster_labels == 0)
test_ff.dcm_ecols[cond_mask].shape

(6840,)

In [182]:
np.where(test_ff.dcm_pairs_idx[cond_mask].tolist() == [[1,2]])

(array([], dtype=int64),)

In [191]:
idx_vals = []
for ii, i in enumerate(test_ff.dcm_pairs_idx[cond_mask].tolist()):
    if i[0][0] == 1 and i[0][1] == 2:
        # print(ii, i)
        idx_vals.append(ii)

In [195]:
test_ff.dcm_pairs_idx[cond_mask][np.array(idx_vals)]
test_ff.dcm_ecols[cond_mask][np.array(idx_vals)].sum()

Array(-0.6017592, dtype=float32)

#  FFs

A FF object should contain all the data needed to fit a potential.
Getting all the jax arrays is slow - FFs should be made once and then reused for preceding fits

Some fits will use Coloumb distances/charges, these are different for each electrostatic model

In [5]:
template_ff = next(read_from_pickle("test_ff.pkl"))

No GPU/TPU found, falling back to CPU. (Set TF_CPP_MIN_LOG_LEVEL=0 and rerun for more info.)


In [222]:
template_ff.data["epol_pairs"] = pair_eel_epol.sum(axis=1)
template_ff.data

,ECOL,KEY,TOTAL,ELEC,VDW,KEY,M_ENERGY,KEY,n_monomers,C_ENERGY,...,p_m_E_tot,C_ENERGY_kcalmol,NBONDS,nb_intE,n,epol_mike,DUMMY,k,ELEC_comp,epol_pairs
test0,-173.022645,test0,24.14023,-86.50353,20.41138,test0,-1527.133263,test0,20,-1527.276175,...,-958281.958376,-958370.381547,-66.09215,-66.09215,0,-197.910658,0,0,-86.50353,-182.982917
test1,-105.096240,test1,25.25058,-55.44306,8.97296,test1,-1527.135387,test1,20,-1527.221635,...,-958282.584930,-958336.157791,-46.47010,-46.47010,1,-113.959761,0,1,-55.44306,-111.406093
test2,-144.567121,test2,24.75756,-76.78875,14.41942,test2,-1527.140946,test2,20,-1527.260418,...,-958285.865743,-958360.493844,-62.36933,-62.36933,2,-160.484782,0,2,-76.78875,-153.110225
test3,-105.558619,test3,21.75948,-60.46748,5.74741,test3,-1527.151295,test3,20,-1527.247454,...,-958292.985606,-958352.359334,-54.72007,-54.72007,3,-118.662845,0,3,-60.46748,-111.548989
test4,-141.957058,test4,21.69034,-77.22292,12.55821,test4,-1527.150321,test4,20,-1527.270835,...,-958292.418141,-958367.030618,-64.66471,-64.66471,4,-155.773808,0,4,-77.22292,-151.839749
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
test495,-156.403481,test495,24.92171,-83.61850,17.54292,test495,-1527.129876,test495,20,-1527.255655,...,-958280.757990,-958357.505436,-66.07558,-66.07558,495,-175.806320,0,495,-83.61850,-166.329665
test496,-126.839243,test496,15.65968,-71.27947,12.74644,test496,-1527.168887,test496,20,-1527.264530,...,-958304.589708,-958363.074582,-58.53303,-58.53303,496,-138.410126,0,496,-71.27947,-133.079412
test497,-160.377711,test497,20.65947,-76.28248,24.76786,test497,-1527.154064,test497,20,-1527.248923,...,-958294.973787,-958353.280867,-51.51462,-51.51462,497,-178.929213,0,497,-76.28248,-170.886865
test498,-154.210967,test498,21.38907,-73.28759,20.35679,test498,-1527.150078,test498,20,-1527.257280,...,-958293.538908,-958358.525003,-52.93080,-52.93080,498,-171.903376,0,498,-73.28759,-164.288222


In [210]:
elec_data = {
    "pc": next(read_from_pickle("data_pc_pbe0.obj.pkl")),
    "kmdcm": next(read_from_pickle("data_kmdcm_pbe0.obj.pkl")),
    "mdcm": next(read_from_pickle("data_mdcm_pbe0.obj.pkl"))
}

In [214]:
elec_keys = ["pc", "kmdcm", "mdcm"]

In [287]:
elec_theory_keys = ["ECOL", "epol_mike", "DUMMY", "epol_pairs"]

#  generate the FF objects:

In [329]:
for typ in ["pairs", "clusters"]:
    # loop through each electrostatic method
    for k in elec_keys:
        print(k, typ)
        data = elec_data[k]
        #  get a fresh template ff
        template_ff = next(read_from_pickle("test_ff.pkl"))
        #  add the epol_pairs data
        template_ff.data["epol_pairs"] = pair_eel_epol.sum(axis=1)
        # add the ELEC column 
        template_ff.data["ELEC"] = data.data["ELEC"]
        
        if typ == "pairs":
            template_ff.intE = "P_intE"
        else:
            template_ff.intE = "intE"
                
        # add the jax data
        jax_data = next(read_from_pickle(f"jax_data_{k}.pkl"))
        template_ff.init_jax_col(jax_data)
        
        template_ff.set_intE()
        template_ff.set_targets()
        
        # verify
        print(template_ff)
        # pickle the ff
        pickle_output(template_ff, f"ff_{k}_pbe0dz_{typ}")


pc pairs
FF: LJ water_cluster ELEC_comp harmonic(jax_coloumb: True)
kmdcm pairs
FF: LJ water_cluster ELEC_comp harmonic(jax_coloumb: True)
mdcm pairs
FF: LJ water_cluster ELEC_comp harmonic(jax_coloumb: True)
pc clusters
FF: LJ water_cluster ELEC_comp harmonic(jax_coloumb: True)
kmdcm clusters
FF: LJ water_cluster ELEC_comp harmonic(jax_coloumb: True)
mdcm clusters
FF: LJ water_cluster ELEC_comp harmonic(jax_coloumb: True)


In [330]:
for typ in ["pairs", "clusters"]:
    # loop through each electrostatic method
    for k in elec_theory_keys:
        print(k, typ)
        #  get a fresh template ff
        template_ff = next(read_from_pickle("test_ff.pkl"))
        
        if typ == "pairs":
            template_ff.intE = "P_intE"
        else:
            template_ff.intE = "intE"
        
        #  add the epol_pairs data
        template_ff.data["epol_pairs"] = pair_eel_epol.sum(axis=1)
        # set the keys
        template_ff.elec = k
        template_ff.intern = "Exact"
        template_ff.set_intE()
        template_ff.set_targets()
        # verify
        print(template_ff)
        # pickle the ff
        pickle_output(template_ff, f"ff_{k}_pbe0dz_{typ}")

ECOL pairs
FF: LJ water_cluster ECOL Exact(jax_coloumb: True)
epol_mike pairs
FF: LJ water_cluster epol_mike Exact(jax_coloumb: True)
DUMMY pairs
FF: LJ water_cluster DUMMY Exact(jax_coloumb: True)
epol_pairs pairs
FF: LJ water_cluster epol_pairs Exact(jax_coloumb: True)
ECOL clusters
FF: LJ water_cluster ECOL Exact(jax_coloumb: True)
epol_mike clusters
FF: LJ water_cluster epol_mike Exact(jax_coloumb: True)
DUMMY clusters
FF: LJ water_cluster DUMMY Exact(jax_coloumb: True)
epol_pairs clusters
FF: LJ water_cluster epol_pairs Exact(jax_coloumb: True)


In [6]:
_c = next(read_from_pickle("ff_mdcm_pbe0dz_clusters.pkl"))
_c.get_loss_coulomb([1])
_c

FF: LJ water_cluster ELEC_comp harmonic intE (jax_coloumb: True)

In [7]:
_p = next(read_from_pickle("ff_pc_pbe0dz_pairs.pkl"))
_p.get_loss_coulomb([1])
_p

FF: LJ water_cluster ELEC_comp harmonic P_intE (jax_coloumb: True)

In [8]:
fit_repeat(_c, 
           10, 
           None,
           loss="jax",
           quiet='true'
          )

/home/boittier/Documents/phd/ff_energy/ff_energy/ff_fit.py:173: OptimizeWarning: Unknown solver options: pgtol
  res = minimize(


Set optimized parameters to FF object, use FF.opt_parm to get the optimized parameters


In [9]:
pd.DataFrame(_c.opt_results).sort_values("fun").iloc[0].x

array([0.25      , 1.81045909, 0.03519921, 0.11794671])

In [385]:
pd.DataFrame(_c.opt_results).sort_values("fun").iloc[0].x

array([0.25000719, 1.72307086, 0.11601976, 0.18228009])

In [10]:
pd.DataFrame(_c.opt_results).sort_values("fun")

,fun,nit,nfev,status,success,message,x,final_simplex
1,18.524570,423,740,0,True,Optimization terminated successfully.,"[0.25, 1.810459086770575, 0.0351992070389889, ...","([[0.25, 1.810459086770575, 0.0351992070389889..."
2,18.524570,623,1073,0,True,Optimization terminated successfully.,"[0.25, 1.8104983083801942, 0.03516987713584115...","([[0.25, 1.8104983083801942, 0.035169877135841..."
5,18.524570,376,665,0,True,Optimization terminated successfully.,"[0.25, 1.8104537127539269, 0.03514921165450044...","([[0.25, 1.8104537127539269, 0.035149211654500..."
7,18.524570,476,856,0,True,Optimization terminated successfully.,"[0.25, 1.8104612611918685, 0.03518830457358059...","([[0.25, 1.8104612611918685, 0.035188304573580..."
8,18.524570,381,663,0,True,Optimization terminated successfully.,"[0.25, 1.8105920672780058, 0.03533880599988127...","([[0.25, 1.8105920672780058, 0.035338805999881..."
10,18.524570,336,587,0,True,Optimization terminated successfully.,"[0.25, 1.810490079604276, 0.035156185712921194...","([[0.25, 1.810490079604276, 0.0351561857129211..."
6,18.524601,233,444,0,True,Optimization terminated successfully.,"[0.2502760092204719, 1.8104756111142652, 0.035...","([[0.2502760092204719, 1.8104756111142652, 0.0..."
3,18.525501,476,857,0,True,Optimization terminated successfully.,"[0.2584953741135563, 1.8094812705302763, 0.031...","([[0.2584953741135563, 1.8094812705302763, 0.0..."
0,129.745026,47,74,0,True,Optimization terminated successfully.,"[0.25, 0.25, 0.00010000000000000003, 0.0001]","([[0.25, 0.25, 0.00010000000000000003, 0.0001]..."
4,523.499207,63,145,0,True,Optimization terminated successfully.,"[0.25, 0.25308332883834483, 0.0001, 0.00010000...","([[0.25, 0.25308332883834483, 0.0001, 0.000100..."


In [366]:
fit_repeat(_p, 
           5, 
           None,
           loss="jax",
           quiet='true'
          )

/home/boittier/Documents/phd/ff_energy/ff_energy/ff_fit.py:173: OptimizeWarning: Unknown solver options: pgtol
  ff.opt_results_df.append(eval(ff.opt_parm))


Set optimized parameters to FF object, use FF.opt_parm to get the optimized parameters


In [367]:
pd.DataFrame(_p.opt_results).sort_values("fun")

,fun,nit,nfev,status,success,message,x,final_simplex
4,3.679285,260,467,0,True,Optimization terminated successfully.,"[0.7396852173580826, 1.7337085825968104, 0.000...","([[0.7396852173580826, 1.7337085825968104, 0.0..."
3,3.838680,339,588,0,True,Optimization terminated successfully.,"[0.26036072300249236, 1.744087746813522, 0.006...","([[0.26036072300249236, 1.744087746813522, 0.0..."
2,3.843210,251,459,0,True,Optimization terminated successfully.,"[0.25079756630910754, 1.7433794613136173, 0.00...","([[0.25079756630910754, 1.7433794613136173, 0...."
5,3.843577,494,862,0,True,Optimization terminated successfully.,"[0.25, 1.7433360461750638, 0.00561815498662891...","([[0.25, 1.7433360461750638, 0.005618154986628..."
1,3.843577,342,593,0,True,Optimization terminated successfully.,"[0.25, 1.7434160220729154, 0.00571847908988658...","([[0.25, 1.7434160220729154, 0.005718479089886..."
0,129.745026,47,74,0,True,Optimization terminated successfully.,"[0.25, 0.25, 0.00010000000000000003, 0.0001]","([[0.25, 0.25, 0.00010000000000000003, 0.0001]..."


In [371]:
_p.get_loss_jax(_c.get_best_parm())

Array(10.536065, dtype=float32)

In [372]:
_c.get_loss_jax(_p.get_best_parm())

Array(24.84817, dtype=float32)

In [331]:
ff_kmdcm_c = next(read_from_pickle("ff_kmdcm_pbe0dz_clusters.pkl"))
ff_kmdcm_c.get_loss_coulomb([1])
ff_kmdcm_c

FF: LJ water_cluster ELEC_comp harmonic(jax_coloumb: True)

In [335]:
pd.DataFrame(ff_kmdcm_c.opt_results).sort_values("fun")

,fun,nit,nfev,status,success,message,x,final_simplex
50,18.260086,381,683,0,True,Optimization terminated successfully.,"[0.25, 1.7817674217092945, 0.01535489644794631...","([[0.25, 1.7817674217092945, 0.015354896447946..."
26,18.260086,342,607,0,True,Optimization terminated successfully.,"[0.25, 1.7819073161264398, 0.01546181958357778...","([[0.25, 1.7819073161264398, 0.015461819583577..."
27,18.260086,222,408,0,True,Optimization terminated successfully.,"[0.25, 1.7817585593237824, 0.01532364099225899...","([[0.25, 1.7817585593237824, 0.015323640992258..."
28,18.260086,470,801,0,True,Optimization terminated successfully.,"[0.25, 1.7818058488731818, 0.01536614545556521...","([[0.25, 1.7818058488731818, 0.015366145455565..."
15,18.260086,428,755,0,True,Optimization terminated successfully.,"[0.25, 1.7818743351469857, 0.01545307041563599...","([[0.25, 1.7818743351469857, 0.015453070415635..."
5,18.260088,445,778,0,True,Optimization terminated successfully.,"[0.25, 1.781870120742144, 0.01537945457111856,...","([[0.25, 1.781870120742144, 0.0153794545711185..."
32,18.260090,388,680,0,True,Optimization terminated successfully.,"[0.25, 1.7819377498823292, 0.01547762635053758...","([[0.25, 1.7819377498823292, 0.015477626350537..."
3,18.261215,308,573,0,True,Optimization terminated successfully.,"[0.2500074752849857, 1.7846180609863116, 0.019...","([[0.2500074752849857, 1.7846180609863116, 0.0..."
13,18.261724,403,734,0,True,Optimization terminated successfully.,"[0.25721024586583124, 1.7810942098332256, 0.01...","([[0.25721024586583124, 1.7810942098332256, 0...."
9,18.265177,208,394,0,True,Optimization terminated successfully.,"[0.25, 1.787843993415433, 0.025056649505161573...","([[0.25, 1.787843993415433, 0.0250566495051615..."


In [336]:
ff_kmdcm_p = next(read_from_pickle("ff_kmdcm_pbe0dz_pairs.pkl"))
ff_kmdcm_p.get_loss_coulomb([1])
ff_kmdcm_p

FF: LJ water_cluster ELEC_comp harmonic(jax_coloumb: True)

In [333]:
fit_repeat(ff_kmdcm_c, 
           50, 
           None,
           loss="jax",
           quiet='true'
          )

/home/boittier/Documents/phd/ff_energy/ff_energy/ff_fit.py:157: OptimizeWarning: Unknown solver options: pgtol
  res = minimize(


Set optimized parameters to FF object, use FF.opt_parm to get the optimized parameters


In [344]:
# ff_kmdcm_p.targets

In [345]:
# ff_kmdcm_c.targets

In [346]:
ff_kmdcm_p.get_loss_jax(ff_kmdcm_c.get_best_parm())**0.5

Array(2.6189387, dtype=float32)

In [347]:
ff_kmdcm_c.get_loss_jax(ff_kmdcm_c.get_best_parm())**0.5

Array(4.2731824, dtype=float32)

In [338]:
ff_kmdcm_p.data

,ECOL,KEY,TOTAL,ELEC,VDW,KEY,M_ENERGY,KEY,n_monomers,C_ENERGY,...,p_m_E_tot,C_ENERGY_kcalmol,NBONDS,nb_intE,n,epol_mike,DUMMY,k,ELEC_comp,epol_pairs
test0,-173.022645,test0,24.14023,-104.54998,20.41138,test0,-1527.133263,test0,20,-1527.276175,...,-958281.958376,-958370.381547,-66.09215,-66.09215,0,-197.910658,0,0,-104.500664,-182.982917
test1,-105.096240,test1,25.25058,-63.45565,8.97296,test1,-1527.135387,test1,20,-1527.221635,...,-958282.584930,-958336.157791,-46.47010,-46.47010,1,-113.959761,0,1,-63.376251,-111.406093
test2,-144.567121,test2,24.75756,-89.34864,14.41942,test2,-1527.140946,test2,20,-1527.260418,...,-958285.865743,-958360.493844,-62.36933,-62.36933,2,-160.484782,0,2,-89.301399,-153.110225
test3,-105.558619,test3,21.75948,-62.57541,5.74741,test3,-1527.151295,test3,20,-1527.247454,...,-958292.985606,-958352.359334,-54.72007,-54.72007,3,-118.662845,0,3,-62.555107,-111.548989
test4,-141.957058,test4,21.69034,-90.29468,12.55821,test4,-1527.150321,test4,20,-1527.270835,...,-958292.418141,-958367.030618,-64.66471,-64.66471,4,-155.773808,0,4,-90.244034,-151.839749
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
test495,-156.403481,test495,24.92171,-95.04093,17.54292,test495,-1527.129876,test495,20,-1527.255655,...,-958280.757990,-958357.505436,-66.07558,-66.07558,495,-175.806320,0,495,-95.007202,-166.329665
test496,-126.839243,test496,15.65968,-75.31144,12.74644,test496,-1527.168887,test496,20,-1527.264530,...,-958304.589708,-958363.074582,-58.53303,-58.53303,496,-138.410126,0,496,-75.305267,-133.079412
test497,-160.377711,test497,20.65947,-90.76807,24.76786,test497,-1527.154064,test497,20,-1527.248923,...,-958294.973787,-958353.280867,-51.51462,-51.51462,497,-178.929213,0,497,-90.769882,-170.886865
test498,-154.210967,test498,21.38907,-87.19819,20.35679,test498,-1527.150078,test498,20,-1527.257280,...,-958293.538908,-958358.525003,-52.93080,-52.93080,498,-171.903376,0,498,-87.210403,-164.288222


In [285]:
ff_kmdcm.elec = "epol_pairs"
ff_kmdcm.elec = "epol_mike"
ff_kmdcm.elec = "ECOL"

ff_kmdcm.intern = "Exact"

ff_kmdcm.set_targets()

print(ff_kmdcm)

fit_repeat(ff_kmdcm, 
           50, 
           "ff_kmdcm",
           loss="jax",
           quiet='true'
          )

FF: LJ water_cluster ECOL Exact(jax_coloumb: True)


/home/boittier/Documents/phd/ff_energy/ff_energy/ff_fit.py:157: OptimizeWarning: Unknown solver options: pgtol
  res = minimize(


Set optimized parameters to FF object, use FF.opt_parm to get the optimized parameters


In [325]:
ff_epol_pairs = next(read_from_pickle("ff_epol_pairs_pbe0dz.pkl"))
ff_epol_mike = next(read_from_pickle("ff_epol_mike_pbe0dz.pkl"))
ff_ECOL = next(read_from_pickle("ff_ECOL_pbe0dz.pkl"))
ff_DUMMY = next(read_from_pickle("ff_DUMMY_pbe0dz.pkl"))

In [302]:
ff_epol_pairs.targets[:10]

Array([ 93.305176,  57.28501 ,  78.14122 ,  51.20867 ,  76.21674 ,
        77.402466, 106.42858 ,  88.34779 ,  76.57274 ,  85.112434],      dtype=float32)

In [304]:
fit_repeat(ff_epol_pairs, 
           10, 
           "ff_epol_pairs",
           loss="jax",
           # quiet='true'
          )

Optimizing LJ parameters...
function: LJ
bounds: ((0.25, 2.5), (0.25, 2.5), (0.0001, 0.5), (0.0001, 0.5))
maxfev: 10000
initial guess: [0.3068317372293462, 1.541438238781364, 0.17697489907317654, 0.41717697788858465]


/home/boittier/Documents/phd/ff_energy/ff_energy/ff_fit.py:157: OptimizeWarning: Unknown solver options: pgtol
  res = minimize(


final_loss_fn:  24.7410945892334
       message: Optimization terminated successfully.
       success: True
        status: 0
           fun: 24.7410945892334
             x: [ 2.500e-01  2.241e+00  3.892e-03  1.774e-02]
           nit: 447
          nfev: 787
 final_simplex: (array([[ 2.500e-01,  2.241e+00,  3.892e-03,  1.774e-02],
                       [ 2.500e-01,  2.241e+00,  3.892e-03,  1.774e-02],
                       ...,
                       [ 2.500e-01,  2.241e+00,  3.892e-03,  1.774e-02],
                       [ 2.500e-01,  2.241e+00,  3.892e-03,  1.774e-02]]), array([ 2.474e+01,  2.474e+01,  2.474e+01,  2.474e+01,
                        2.474e+01]))
Optimizing LJ parameters...
function: LJ
bounds: ((0.25, 2.5), (0.25, 2.5), (0.0001, 0.5), (0.0001, 0.5))
maxfev: 10000
initial guess: [2.2191924994046364, 1.0302935668864444, 0.31249818059887785, 0.24529535639433622]
final_loss_fn:  25.292383193969727
       message: Optimization terminated successfully.
       success: T

In [303]:
ff_epol_mike.targets[:10]

Array([108.232925,  59.838684,  85.51578 ,  58.322525,  80.150795,
        86.40199 , 110.762474,  92.635544,  78.44148 ,  91.89037 ],      dtype=float32)

In [324]:
fit_repeat(ff_epol_mike, 
           10, 
           "ff_epol_mike",
           loss="jax",
           # quiet='true'
          )

Optimizing LJ parameters...
function: LJ
bounds: ((0.25, 2.5), (0.25, 2.5), (0.0001, 0.5), (0.0001, 0.5))
maxfev: 10000
initial guess: [1.8748889720200097, 1.7173266969707968, 0.3097800904447254, 0.3173455313938197]


/home/boittier/Documents/phd/ff_energy/ff_energy/ff_fit.py:157: OptimizeWarning: Unknown solver options: pgtol
  res = minimize(


final_loss_fn:  19.35767364501953
       message: Optimization terminated successfully.
       success: True
        status: 0
           fun: 19.35767364501953
             x: [ 2.500e-01  2.153e+00  1.016e-04  3.351e-02]
           nit: 354
          nfev: 650
 final_simplex: (array([[ 2.500e-01,  2.153e+00,  1.016e-04,  3.351e-02],
                       [ 2.500e-01,  2.153e+00,  1.016e-04,  3.351e-02],
                       ...,
                       [ 2.500e-01,  2.153e+00,  1.016e-04,  3.351e-02],
                       [ 2.500e-01,  2.153e+00,  1.016e-04,  3.351e-02]]), array([ 1.936e+01,  1.936e+01,  1.936e+01,  1.936e+01,
                        1.936e+01]))
Optimizing LJ parameters...
function: LJ
bounds: ((0.25, 2.5), (0.25, 2.5), (0.0001, 0.5), (0.0001, 0.5))
maxfev: 10000
initial guess: [2.1711198333555393, 1.0427898089307372, 0.22010334291152134, 0.33981630318972095]
final_loss_fn:  16.82760238647461
       message: Optimization terminated successfully.
       success: 

In [322]:
# ff_epol_mike.set_intE(pairs=True)

In [323]:
ff_epol_mike.data[["intE", "P_intE"]]

,intE,P_intE
test0,-77.478222,-77.478222
test1,-53.285800,-53.285800
test2,-70.885989,-70.885989
test3,-58.662892,-58.662892
test4,-73.371184,-73.371184
...,...,...
test495,-72.611238,-72.611238
test496,-61.118914,-61.118914
test497,-57.960443,-57.960443
test498,-65.470853,-65.470853


In [308]:
ff_ECOL.targets[:10]

Array([83.34491 , 50.97516 , 69.598114, 45.2183  , 66.33405 , 68.995224,
       97.11085 , 79.32822 , 66.247665, 74.90504 ], dtype=float32)

In [309]:
fit_repeat(ff_ECOL, 
           10, 
           "ff_ECOL",
           loss="jax",
           # quiet='true'
          )

Optimizing LJ parameters...
function: LJ
bounds: ((0.25, 2.5), (0.25, 2.5), (0.0001, 0.5), (0.0001, 0.5))
maxfev: 10000
initial guess: [2.299435178534382, 1.2544748014153848, 0.1120325742410918, 0.07700198284503466]


/home/boittier/Documents/phd/ff_energy/ff_energy/ff_fit.py:157: OptimizeWarning: Unknown solver options: pgtol
  res = minimize(


final_loss_fn:  22.38670539855957
       message: Optimization terminated successfully.
       success: True
        status: 0
           fun: 22.38670539855957
             x: [ 2.500e-01  2.215e+00  3.609e-02  1.705e-02]
           nit: 408
          nfev: 740
 final_simplex: (array([[ 2.500e-01,  2.215e+00,  3.609e-02,  1.705e-02],
                       [ 2.500e-01,  2.215e+00,  3.609e-02,  1.705e-02],
                       ...,
                       [ 2.500e-01,  2.215e+00,  3.609e-02,  1.705e-02],
                       [ 2.500e-01,  2.215e+00,  3.609e-02,  1.705e-02]]), array([ 2.239e+01,  2.239e+01,  2.239e+01,  2.239e+01,
                        2.239e+01]))
Optimizing LJ parameters...
function: LJ
bounds: ((0.25, 2.5), (0.25, 2.5), (0.0001, 0.5), (0.0001, 0.5))
maxfev: 10000
initial guess: [1.1291077395011468, 1.6816845147542387, 0.07250531950924333, 0.4479902384450863]
final_loss_fn:  110.92178344726562
       message: Optimization terminated successfully.
       success: 

In [311]:
ff_DUMMY.targets[:10]

Array([-89.677734, -54.12108 , -74.96901 , -60.340317, -75.62301 ,
       -64.597565, -62.311775, -54.60139 , -62.371445, -66.50143 ],      dtype=float32)

In [312]:
fit_repeat(ff_DUMMY, 
           10, 
           "ff_DUMMY",
           loss="jax",
           # quiet='true'
          )

Optimizing LJ parameters...
function: LJ
bounds: ((0.25, 2.5), (0.25, 2.5), (0.0001, 0.5), (0.0001, 0.5))
maxfev: 10000
initial guess: [0.8649110489655808, 0.6599718074239327, 0.13827743984063107, 0.4141645771097512]


/home/boittier/Documents/phd/ff_energy/ff_energy/ff_fit.py:157: OptimizeWarning: Unknown solver options: pgtol
  res = minimize(


final_loss_fn:  872.065673828125
       message: Optimization terminated successfully.
       success: True
        status: 0
           fun: 872.065673828125
             x: [ 1.117e+00  7.798e-01  5.000e-01  5.000e-01]
           nit: 339
          nfev: 590
 final_simplex: (array([[ 1.117e+00,  7.798e-01,  5.000e-01,  5.000e-01],
                       [ 1.117e+00,  7.798e-01,  5.000e-01,  5.000e-01],
                       ...,
                       [ 1.117e+00,  7.798e-01,  5.000e-01,  5.000e-01],
                       [ 1.117e+00,  7.798e-01,  5.000e-01,  5.000e-01]]), array([ 8.721e+02,  8.721e+02,  8.721e+02,  8.721e+02,
                        8.721e+02]))
Optimizing LJ parameters...
function: LJ
bounds: ((0.25, 2.5), (0.25, 2.5), (0.0001, 0.5), (0.0001, 0.5))
maxfev: 10000
initial guess: [0.8796171738736194, 2.3972234927158738, 0.3779065782768344, 0.17898745074176098]
final_loss_fn:  872.1819458007812
       message: Optimization terminated successfully.
       success: Tru

In [313]:
from ff_energy.plot import plot_energy_MSE, plot_ff_fit
plot_ff_fit

In [269]:
pd.DataFrame(ff_kmdcm.opt_results).sort_values("fun")

,fun,nit,nfev,status,success,message,x,final_simplex
108,18.260086,258,474,0,True,Optimization terminated successfully.,"[0.25, 1.7818641352210869, 0.01542444210291486...","([[0.25, 1.7818641352210869, 0.015424442102914..."
101,18.260086,406,708,0,True,Optimization terminated successfully.,"[0.25, 1.781796153086285, 0.015343639151067668...","([[0.25, 1.781796153086285, 0.0153436391510676..."
86,18.260086,257,475,0,True,Optimization terminated successfully.,"[0.25, 1.7818619341165665, 0.01544404202753632...","([[0.25, 1.7818619341165665, 0.015444042027536..."
24,18.260086,398,699,0,True,Optimization terminated successfully.,"[0.25, 1.7818034776255884, 0.01537512718831042...","([[0.25, 1.7818034776255884, 0.015375127188310..."
37,18.260088,339,632,0,True,Optimization terminated successfully.,"[0.25, 1.781758063291883, 0.015381010490841058...","([[0.25, 1.781758063291883, 0.0153810104908410..."
...,...,...,...,...,...,...,...,...
10,407.631012,39,62,0,True,Optimization terminated successfully.,"[0.25, 0.25, 0.0001, 0.5]","([[0.25, 0.25, 0.0001, 0.5], [0.25, 0.25, 0.00..."
49,407.631012,48,83,0,True,Optimization terminated successfully.,"[0.25, 0.25, 0.00010000000000000002, 0.5]","([[0.25, 0.25, 0.00010000000000000002, 0.5], [..."
68,407.798157,82,124,0,True,Optimization terminated successfully.,"[0.25, 0.25, 0.5, 0.00010000000000000002]","([[0.25, 0.25, 0.5, 0.00010000000000000002], [..."
107,407.798157,43,71,0,True,Optimization terminated successfully.,"[0.25, 0.25, 0.5, 0.0001]","([[0.25, 0.25, 0.5, 0.0001], [0.25, 0.25, 0.5,..."


In [255]:
ff_kmdcm.targets

Array([16.126808  ,  9.88279   , 14.720535  ,  3.2016792 , 15.682198  ,
       15.466427  , 32.20911   , 23.919983  , 15.386822  , 17.434814  ,
       10.65123   , 29.235786  , 16.692871  , 24.447548  , 14.826561  ,
       21.564262  , 23.640518  , 23.46782   , 25.8525    , 38.21783   ,
       30.016197  , 29.034184  , 43.176765  , 21.497452  , 25.969742  ,
       23.101135  , 12.628902  , 14.052521  ,  6.4274673 , 23.344791  ,
       16.25756   , 22.185005  , 12.725586  , 21.91452   ,  9.985352  ,
       16.410912  , 24.354767  ,  8.255455  , 24.46671   , 19.860203  ,
       15.116035  , 18.606457  , 23.740585  ,  9.39035   , 10.648331  ,
       13.825241  , 10.93055   , 16.010448  , 20.600391  , 20.077702  ,
        9.749798  , 10.1818695 , 20.09805   , 24.867603  , 15.050064  ,
       12.055725  ,  9.374664  , 19.131725  , 19.679367  , 26.610458  ,
       26.287666  , 27.940292  , 18.47274   , 24.447327  , 16.974987  ,
       20.653503  , 12.34922   , 21.967125  , 25.588913  , 13.71

In [212]:
test_ff = load_ff("test", 
        "water_cluster", 
        pk = "data_pc_pbe0.obj.pkl",
        intern="harmonic",
        elec="ECOL"
       )
test_ff, data = test_ff

Pickled structures/dists found:  ['water_cluster.pkl.pkl', 'water_dimer.pkl.pkl', 'water_dimer.pkl', 'structures.pkl', 'dists.pkl', 'water_cluster.pkl']
Loading pickled distances/structure: water_cluster
loading data from pickle: data_pc_pbe0.obj.pkl


KeyboardInterrupt: 

In [199]:
test_ff = next(read_from_pickle("test_ff.pkl"))

In [231]:
data_kmdcm = next(read_from_pickle("data_kmdcm_pbe0.obj.pkl"))

data_kmdcm.data

,ECOL,KEY,TOTAL,ELEC,VDW,KEY,M_ENERGY,KEY,n_monomers,C_ENERGY,n_pairs,intE,P_intE,m_E_tot,p_m_E_tot,C_ENERGY_kcalmol,NBONDS,nb_intE
test467,-113.436686,test467,22.56645,-64.08560,14.82881,test467,-1527.154217,test467,20,-1527.236629,190,-51.714066,-50.297664,-958293.852586,-958295.211607,-958345.566652,-49.25679,-49.25679
test263,-159.213355,test263,21.29141,-92.76477,20.54380,test263,-1527.156016,test263,20,-1527.258259,190,-64.157783,-66.601357,-958294.981314,-958296.144719,-958359.139096,-72.22097,-72.22097
test35,-138.530827,test35,23.43646,-87.37824,11.05221,test35,-1527.143158,test35,20,-1527.258584,189,-72.430230,-65.790457,-958286.913137,-958288.376038,-958359.343368,-76.32603,-76.32603
test223,-85.840765,test223,24.37323,-47.13218,6.72946,test223,-1527.147556,test223,20,-1527.203106,190,-34.857221,-39.243367,-958289.673128,-958290.449874,-958324.530348,-40.40272,-40.40272
test254,-157.431989,test254,27.07465,-90.75359,21.85417,test254,-1527.135568,test254,20,-1527.237145,188,-63.739697,-57.830990,-958282.150371,-958283.041933,-958345.890067,-68.89942,-68.89942
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
test100,-120.244324,test100,25.63059,-66.67887,19.45476,test100,-1527.141372,test100,20,-1527.210157,190,-43.162995,-42.125801,-958285.792204,-958287.528319,-958328.955198,-47.22411,-47.22411
test304,-123.903711,test304,23.20412,-72.62700,17.28253,test304,-1527.150985,test304,20,-1527.239633,189,-55.626955,-54.710834,-958291.824384,-958293.116885,-958347.451339,-55.34447,-55.34447
test327,-195.846922,test327,21.75196,-117.28678,31.12436,test327,-1527.154084,test327,20,-1527.300559,190,-91.913853,-79.340573,-958293.768884,-958295.335949,-958385.682737,-86.16242,-86.16242
test185,-114.232038,test185,25.65796,-69.10066,11.20177,test185,-1527.144999,test185,20,-1527.232387,190,-54.835874,-54.451833,-958288.068584,-958289.053297,-958342.904458,-57.89889,-57.89889


In [206]:
test_ff.data["ECOL"] = data_kmdcm.data["ECOL"]

In [208]:
test_ff.data

,ECOL,KEY,TOTAL,ELEC,VDW,KEY,M_ENERGY,KEY,n_monomers,C_ENERGY,...,m_E_tot,p_m_E_tot,C_ENERGY_kcalmol,NBONDS,nb_intE,n,epol_mike,DUMMY,k,ELEC_comp
test0,-173.022645,test0,24.14023,-86.50353,20.41138,test0,-1527.133263,test0,20,-1527.276175,...,-958280.703813,-958281.958376,-958370.381547,-66.09215,-66.09215,0,-197.910658,0,0,-86.50353
test1,-105.096240,test1,25.25058,-55.44306,8.97296,test1,-1527.135387,test1,20,-1527.221635,...,-958282.036711,-958282.584930,-958336.157791,-46.47010,-46.47010,1,-113.959761,0,1,-55.44306
test2,-144.567121,test2,24.75756,-76.78875,14.41942,test2,-1527.140946,test2,20,-1527.260418,...,-958285.524837,-958285.865743,-958360.493844,-62.36933,-62.36933,2,-160.484782,0,2,-76.78875
test3,-105.558619,test3,21.75948,-60.46748,5.74741,test3,-1527.151295,test3,20,-1527.247454,...,-958292.019016,-958292.985606,-958352.359334,-54.72007,-54.72007,3,-118.662845,0,3,-60.46748
test4,-141.957058,test4,21.69034,-77.22292,12.55821,test4,-1527.150321,test4,20,-1527.270835,...,-958291.407609,-958292.418141,-958367.030618,-64.66471,-64.66471,4,-155.773808,0,4,-77.22292
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
test495,-156.403481,test495,24.92171,-83.61850,17.54292,test495,-1527.129876,test495,20,-1527.255655,...,-958278.578831,-958280.757990,-958357.505436,-66.07558,-66.07558,495,-175.806320,0,495,-83.61850
test496,-126.839243,test496,15.65968,-71.27947,12.74644,test496,-1527.168887,test496,20,-1527.264530,...,-958303.057842,-958304.589708,-958363.074582,-58.53303,-58.53303,496,-138.410126,0,496,-71.27947
test497,-160.377711,test497,20.65947,-76.28248,24.76786,test497,-1527.154064,test497,20,-1527.248923,...,-958293.756804,-958294.973787,-958353.280867,-51.51462,-51.51462,497,-178.929213,0,497,-76.28248
test498,-154.210967,test498,21.38907,-73.28759,20.35679,test498,-1527.150078,test498,20,-1527.257280,...,-958291.255270,-958293.538908,-958358.525003,-52.93080,-52.93080,498,-171.903376,0,498,-73.28759


In [48]:
test_ff.get_coulomb(1)
set(test_ff.dcm_c1s.tolist())

{-0.3034000098705292, 0.1054999977350235, 0.19789999723434448}

In [72]:
test_ff.dcm_ecols

Array([ 3.340709  ,  1.6438198 , -4.480503  , ..., -1.8386152 ,
        1.2585741 ,  0.63554394], dtype=float32)

In [117]:
test_ff.eval_coulomb(1)

Array([ 3.340709  ,  1.6438199 , -4.480503  , ..., -1.8386152 ,
        1.2585741 ,  0.63554394], dtype=float32)

In [113]:
# test_ff.elec = "ELEC"
test_ff.elec = "ELEC_comp"

test_ff.set_targets()
print(test_ff.targets[:10])
print(test_ff.data[test_ff.elec][:10])
print(test_ff.data["intE"][:10])

[16.077492   9.8033905 14.673294   3.1813774 15.631554  15.461651
 32.179363  25.048508  15.366566  17.394058 ]
test0   -104.500664
test1    -63.376251
test2    -89.301399
test3    -62.555107
test4    -90.244034
test5    -78.528320
test6    -92.474442
test7    -78.314598
test8    -77.122337
test9    -82.941277
Name: ELEC_comp, dtype: float32
test0   -88.423171
test1   -53.572861
test2   -74.628101
test3   -59.373728
test4   -74.612477
test5   -63.066668
test6   -60.295078
test7   -53.266092
test8   -61.755771
test9   -65.547217
Name: intE, dtype: float64


In [96]:
test_ff.data["ELEC_comp"] 

test0     -104.500664
test1      -63.376251
test2      -89.301399
test3      -62.555107
test4      -90.244034
              ...    
test495    -95.007202
test496    -75.305267
test497    -90.769882
test498    -87.210403
test499    -92.603050
Name: ELEC_comp, Length: 500, dtype: float32

In [110]:
test_ff.get_loss_coulomb([1])

Array(2215976.5, dtype=float32)

In [109]:
test_ff.get_coulomb(1)

Array([-104.50066 ,  -63.37625 ,  -89.3014  ,  -62.555107,  -90.24403 ,
        -78.52832 ,  -92.47444 ,  -78.3146  ,  -77.12234 ,  -82.94128 ,
        -62.94162 ,  -80.44895 ,  -83.78231 ,  -93.72382 ,  -89.5733  ,
        -87.799614,  -88.30298 ,  -91.08435 , -101.89189 ,  -91.036385,
        -93.7976  ,  -90.596405,  -88.27305 ,  -68.85179 ,  -92.42027 ,
        -95.55438 ,  -65.20173 ,  -51.60432 ,  -31.270267,  -64.36589 ,
        -72.16843 ,  -75.02931 ,  -68.25783 ,  -96.667816,  -75.1862  ,
        -83.20317 ,  -82.521736,  -73.63057 ,  -58.877853,  -68.60941 ,
        -51.701195,  -76.08404 ,  -67.55745 ,  -75.73437 ,  -46.40029 ,
        -57.727802,  -52.369377,  -51.69393 ,  -84.18983 ,  -81.97436 ,
        -73.6404  ,  -84.35237 ,  -72.27201 ,  -81.359795,  -77.192245,
        -62.42471 ,  -75.337494,  -67.13616 ,  -85.83446 ,  -81.11418 ,
       -118.955284,  -79.52722 ,  -77.90353 ,  -57.135178,  -55.85014 ,
       -121.669945,  -84.530106,  -74.32834 ,  -96.81129 ,  -59.

In [126]:
# test_ff.eval_coulomb_nb([1])

In [124]:
test_ff.p = test_ff.get_best_parm()

In [81]:
# test_ff.eval_lj_coulomb(test_ff.get_best_parm())

In [128]:
fit_repeat(test_ff, 1, "test-ff", bounds=[(0.9,1.4)], loss="ecol")

Optimizing LJ parameters...
function: LJ
bounds: [(0.9, 1.4)]
maxfev: 10000
initial guess: [1.2717929874715694]


/home/boittier/Documents/phd/ff_energy/ff_energy/ff_fit.py:157: OptimizeWarning: Unknown solver options: pgtol
  res = minimize(


final_loss_fn:  18.221405029296875
       message: Optimization terminated successfully.
       success: True
        status: 0
           fun: 18.221405029296875
             x: [ 1.001e+00]
           nit: 26
          nfev: 61
 final_simplex: (array([[ 1.001e+00],
                       [ 1.001e+00]]), array([ 1.822e+01,  1.822e+01]))
Set optimized parameters to FF object, use FF.opt_parm to get the optimized parameters


IndexError: index 2 is out of bounds for axis 0 with size 1

In [71]:
# test_ff.eval_best_parm()
test_ff.data["ELEC_comp"]

test0     -26.124409
test1     -15.843583
test2     -22.324703
test3     -15.638281
test4     -22.560249
             ...    
test495   -23.750986
test496   -18.825729
test497   -22.691795
test498   -21.801907
test499   -23.150061
Name: ELEC_comp, Length: 500, dtype: float32

In [46]:
jax_data_kmdcm = next(read_from_pickle("jax_data_kmdcm.pkl"))

In [49]:
test_ff.init_jax_col(jax_data_kmdcm)

In [27]:
# test_ff.cluster_labels

In [19]:
test_ff
test_ff.p

Array([1.764   , 0.25    , 0.1687  , 0.006871], dtype=float32)

In [18]:
# x0 = test_ff.get_best_parm() #[2.1801901325157633, 0.9689078330468776, 0.3951433427330229, 0.26583564356898853, 1.34]
x0 = [1.343405968282677, 1.3305735932671763, 0.4156146497222247, 0.05713119128733019, 1.]
sig_bound = (0.25, 2.5)
ep_bound = (0.0001, 0.5)
bounds = ((sig_bound), (sig_bound), (ep_bound), (ep_bound), (1.0,1.5))

fit_func(test_ff, x0, bounds=bounds, loss='ecol')

Optimizing LJ parameters...
function: LJ
bounds: ((0.25, 2.5), (0.25, 2.5), (0.0001, 0.5), (0.0001, 0.5), (1.0, 1.5))
maxfev: 10000
initial guess: [1.343405968282677, 1.3305735932671763, 0.4156146497222247, 0.05713119128733019, 1.0]


/home/boittier/Documents/phd/ff_energy/ff_energy/ff_fit.py:158: OptimizeWarning: Unknown solver options: pgtol
  res = minimize(


final_loss_fn:  1334039.875
       message: Optimization terminated successfully.
       success: True
        status: 0
           fun: 1334039.875
             x: [ 2.500e+00  1.044e+00  3.847e-01  5.078e-02  1.000e+00]
           nit: 39
          nfev: 160
 final_simplex: (array([[ 2.500e+00,  1.044e+00, ...,  5.078e-02,
                         1.000e+00],
                       [ 2.500e+00,  1.044e+00, ...,  5.078e-02,
                         1.000e+00],
                       ...,
                       [ 2.500e+00,  1.044e+00, ...,  5.078e-02,
                         1.000e+00],
                       [ 2.500e+00,  1.044e+00, ...,  5.078e-02,
                         1.000e+00]]), array([ 1.334e+06,  1.334e+06,  1.334e+06,  1.334e+06,
                        1.334e+06,  1.334e+06]))


       message: Optimization terminated successfully.
       success: True
        status: 0
           fun: 1334039.875
             x: [ 2.500e+00  1.044e+00  3.847e-01  5.078e-02  1.000e+00]
           nit: 39
          nfev: 160
 final_simplex: (array([[ 2.500e+00,  1.044e+00, ...,  5.078e-02,
                         1.000e+00],
                       [ 2.500e+00,  1.044e+00, ...,  5.078e-02,
                         1.000e+00],
                       ...,
                       [ 2.500e+00,  1.044e+00, ...,  5.078e-02,
                         1.000e+00],
                       [ 2.500e+00,  1.044e+00, ...,  5.078e-02,
                         1.000e+00]]), array([ 1.334e+06,  1.334e+06,  1.334e+06,  1.334e+06,
                        1.334e+06,  1.334e+06]))

In [107]:
test_ff.data[["intE"]]

,intE
test0,-88.423171
test1,-53.572861
test2,-74.628101
test3,-59.373728
test4,-74.612477
...,...
test495,-76.747446
test496,-58.484874
test497,-58.307079
test498,-64.986095


In [34]:
# test_ff.eval_lj_coulomb(x0)
test_ff.targets

Array([-66.92437 , -40.534542, -56.25623 , -46.504326, -56.046722,
       -46.91107 , -41.2704  , -37.154556, -45.88945 , -48.48375 ,
       -39.36499 , -34.64701 , -49.871586, -49.99745 , -56.357147,
       -48.154987, -46.514446, -48.85701 , -55.0724  , -34.09836 ,
       -44.494762, -42.93338 , -26.904875, -33.204956, -47.46854 ,
       -52.829918, -39.15078 , -26.174866, -18.389687, -28.607162,
       -41.0792  , -37.4188  , -41.51284 , -55.45493 , -49.70831 ,
       -53.850037, -41.18997 , -50.25609 , -22.273453, -34.623062,
       -25.958157, -41.818016, -29.906507, -50.799805, -26.192917,
       -32.05584 , -30.638247, -25.45947 , -46.29307 , -45.07351 ,
       -48.79098 , -56.864937, -37.341705, -39.79544 , -46.245342,
       -42.181835, -50.495377, -34.21025 , -48.53238 , -37.83159 ,
       -68.15353 , -35.24983 , -43.40808 , -20.871304, -27.1768  ,
       -76.03003 , -54.825592, -37.090107, -51.344788, -33.731537,
       -39.378487, -38.673504, -23.620745, -18.69637 , -37.737